# Measures of Dispersion

**Dispersion** measures how spread out of a set of data is. This is especially important in finance because one of the main ways *risk* is measured is in how spread out *returns* have been historically. If returns have been very tight around a central value, then we have less reason to worry. If returns have been all over the place, that is risky.

Data with low dispersion if heavily clustered around the mean, while data with high dispersion indicates many very large and very small values.

Let's generate an array of random integers to work with.

In [3]:
import numpy as np
np.random.seed(42)

In [5]:
# Generate 20 random integers < 100
X = np.random.randint(100, size=20)

# Sort them
X = np.sort(X)
print(f"X: {X}")

# The average
mu = np.mean(X)
print(f"Mean of X: {mu}")

X: [ 1 14 20 21 32 41 46 48 57 58 59 59 61 61 63 75 79 88 90 91]
Mean of X: 53.2


## Range

**Range** is simply the difference between the maximum and the minimum values in the dataset. Not surprisingly, it is very sensitive to outliers. We'll use `numpy`'s peak to peak (`ptp`) function for this.

In [6]:
print(f"Range of X: {np.ptp(X)}")

Range of X: 90


## Mean Absolute Deviation (MAD)

The **mean absolute deviation** is the average of the distances of observations from the arithmetic mean. We use the absolute value of the deviation, so that 5 above the mean and 5 above the mean both contribute 5, because otherwise the deviations always sum to 0.

$$ MAD = \frac{\sum_{i=1}^n |X_i - \mu|}{n} $$

where $n$ is the number of observations and $\mu$ is their mean.

In [8]:
abs_dispersion = [np.abs(mu - x) for x in X]
MAD = np.sum(abs_dispersion) / len(abs_dispersion)
print(f"Mean absolute deviation of X: {MAD}")

Mean absolute deviation of X: 20.26


## Variance and Standard Deviation

The **variance** $\sigma^2$ is defined as the average of the squared deviations around the mean

$$ \sigma^2 = \frac{\sum_{i=1}^n (X_i - \mu)^2}{n} $$

This is sometimes more convenient than the mean absolute deviation because the absolute value is not differentiable, while squaring is smooth, and some optimization algorithms rely on differentiability.

**Standard deviation** is defined as the square root of the variance, $\sigma$, and it is the easier of the two to interpret because it is in the same units as the observations.

In [9]:
print(f"Variance of X: {np.var(X)}")
print(f"Standard deviation of X: {np.std(X)}")

Variance of X: 626.76
Standard deviation of X: 25.03517525403008


One way to interpret standard deviation is by referring to *Chebyshev's inequality*. This tells us that the proportion of samples within $k$ standard deviations (that is, within a distance of $k \cdot$ standard deviation) of the mean is at least $1 - 1/k^2$ for all $k > 1$.

Let's check that if this is true for our dataset.

In [11]:
k = 1.25
dist = k * np.std(X)
l = [x for x in X if abs(x - mu) <= dist]
print(f"Observations within {k} standard deviations of mean: {l}")
print(f"Confirming that {len(l) / len(X)} > {1 - 1/k**2}")

Observations within 1.25 standard deviations of mean: [32, 41, 46, 48, 57, 58, 59, 59, 61, 61, 63, 75, 79]
Confirming that 0.65 > 0.36


The bound given by Chebyshev's inequality seems fairly loose in this case. This bound is rarely strict, but it is useful because it holds for all datasets and distributions.

## Semivariance and semideviation

Although variance and standard deviation tell us how volatile a quantity is, they do not differentiate between deviations upwawrd and deviations downward. Often, such as in the case of returns on an asset, we are more worried about deviations downward. This is addressed by semivariance and semideviation, which only count the observations that fall below the mean. **Semivariance** is defined as

$$ \frac{\sum_{X_i < \mu} (X_i - \mu)^2}{n_<} $$

where $n_<$ is the number of observations which are smaller than the mean. **Semideviation** is the square root of the semivariance.


In [12]:
# Since there is no built-in semi-deviation, we'll compute this ourselves
lows = [e for e in X if e <= mu]
semivar = np.sum((lows - mu)**2 / len(lows))
print(f"Semivariance of X: {semivar}")
print(f"Semideviation of X: {np.sqrt(semivar)}")

Semivariance of X: 884.7150000000001
Semideviation of X: 29.744159090483635


A related notion is **target semivariance** (and **target semideviation**), where we average the distance from a target of values which fall below that target:
$$ \frac{\sum_{X_i < B} (X_i - B)^2}{n_{<B}} $$


In [13]:
B = 19
lows_B = [e for e in X if e < B]
semivar_B = sum(map(lambda x: (x - B)**2, lows_B)) / len(lows_B)

print(f"Target semivariance of X: {semivar_B}")
print(f"Target semideviation of X: {np.sqrt(semivar_B)}")

Target semivariance of X: 174.5
Target semideviation of X: 13.209844813622906


# These are Only Estimates

All of these computations will give us *sample statistics*, that is standard deviation of a sample of data. Whether or not this reflects the current true population standard deviation is not always obvious, and more effort has to be put into determining that. This is especially problematic in finance because all data are time series and the mean and variance may change over time. There are many different techniques and subtleties here than we think.

*In general do not assume that because something is true of our sample, it will remain true going forward.*